In [2]:
import pandas as pd
import numpy as np

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import random_split

In [3]:
df_raw_train = pd.read_csv('data/train.csv')
df_raw_val = pd.read_csv("data/test.csv")

In [4]:
label = 'SalePrice'

# Drop the 'Id' column from the train and test set
features = pd.concat(
    (df_raw_train.drop(["Id", label], axis=1),
     df_raw_val.drop(["Id"], axis=1)))

# Standardize numerical columns
numeric_features = features.dtypes[features.dtypes!='object'].index
features[numeric_features] = features[numeric_features].apply(
        lambda x: (x - x.mean()) / (x.std()))

 # Replace NAN numerical features by 0
features[numeric_features] = features[numeric_features].fillna(0)

# Replace discrete features by one-hot encoding
features = pd.get_dummies(features, dummy_na=True)

train_df = features[:df_raw_train.shape[0]].copy()
val_df = features[df_raw_train.shape[0]:].copy()

In [5]:
# structure of training data, spliting it into X and y
class dataset(Dataset):
    def __init__(self,X,Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return [self.X[idx], self.Y[idx]]

In [6]:
X = torch.tensor(train_df.values, dtype=torch.float)
y = torch.tensor(df_raw_train[label].values, dtype=torch.float).unsqueeze(1)

train_set, test_set = random_split(dataset(X,y), [0.8, 0.2])

train_dataloader = DataLoader(train_set, batch_size=64)
test_dataloader = DataLoader(test_set, batch_size=64)

In [7]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [8]:
class Net(nn.Module):
    def __init__(self, D_in, H1, H2, H3, D_out):
        super(Net, self).__init__()

        self.linear1 = nn.Linear(D_in, H1)
        self.linear2 = nn.Linear(H1, H2)
        self.linear3 = nn.Linear(H2, H3)
        self.linear4 = nn.Linear(H3, D_out)

    def forward(self, x):
        y_pred = self.linear1(x).clamp(min=0)                   #clamp(min=0) set y_pred = 0 if y_pred < 0 since price is always >= 0
        y_pred = self.linear2(y_pred).clamp(min=0)
        y_pred = self.linear3(y_pred).clamp(min=0)
        y_pred = self.linear4(y_pred)
        return y_pred

H1, H2, H3 = 500, 1000, 200
D_in, D_out = 331, 1
model = Net(D_in, H1, H2, H3, D_out).to(device)

In [9]:
# Root mean squared log loss
class RMSLELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss(reduction="sum")

    def forward(self, pred, actual):
        return torch.sqrt(self.mse(torch.log(pred + 1), torch.log(actual + 1)))

loss_fn = RMSLELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

#       if batch % 5 == 0:
#           loss, current = loss.item(), (batch + 1) * len(X)
#           print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [11]:
def test(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss = 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
#   print(f"Test Error: \nAvg loss: {test_loss:>8f} \n")

In [12]:
epochs = 2000
for t in range(epochs):
#   print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

In [14]:
preds = model(torch.tensor(val_df.values, dtype=torch.float).to(device))
submission = pd.DataFrame({'Id':df_raw_val.Id,
                           'SalePrice':preds.cpu().squeeze(1).detach().numpy()})
submission.to_csv('submission2.csv', index=False)